# AI Prompt Security Detection - Phase 1: Data Collection

This notebook guides you through **Phase 1** of the project.

## Objectives
1. ✅ Download malicious and benign prompt datasets
2. ✅ Clean and label the data
3. ✅ Save processed data to `data/processed/prompts.csv`

## Labels
- `0` = Benign
- `1` = Prompt Injection
- `2` = Jailbreak

## 1. Setup and Imports

In [ ]:
# Install requirements if not already installed
# !pip install -r ../requirements.txt

In [ ]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.append('../src')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✅ Imports successful")

## 2. Download Datasets

⚠️ **Note**: This may take several minutes and requires internet connection.

In [ ]:
# Run the download script
from download_datasets import main as download_main

download_main()

## 3. Clean and Label Data

In [ ]:
# Run the cleaning script
from data_cleaning import DataCleaner

cleaner = DataCleaner()
cleaner.run()

## 4. Explore Processed Data

In [ ]:
# Load processed data
df = pd.read_csv('../data/processed/prompts.csv')

print(f"Total samples: {len(df):,}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Label distribution
label_map = {0: 'Benign', 1: 'Injection', 2: 'Jailbreak'}
df['label_name'] = df['label'].map(label_map)

print("\n📊 Label Distribution:")
print(df['label_name'].value_counts())

# Visualize
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='label_name', order=['Benign', 'Injection', 'Jailbreak'])
plt.title('Distribution of Prompt Labels', fontsize=14, fontweight='bold')
plt.xlabel('Label')
plt.ylabel('Count')
plt.xticks(rotation=0)
for container in plt.gca().containers:
    plt.gca().bar_label(container)
plt.tight_layout()
plt.show()

In [ ]:
# Text length distribution
df['text_length'] = df['text'].str.len()

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df.groupby('label_name')['text_length'].hist(alpha=0.6, bins=50, legend=True)
plt.xlabel('Text Length (characters)')
plt.ylabel('Frequency')
plt.title('Text Length Distribution by Label')
plt.legend(['Benign', 'Injection', 'Jailbreak'])

plt.subplot(1, 2, 2)
df.boxplot(column='text_length', by='label_name', figsize=(12, 5))
plt.xlabel('Label')
plt.ylabel('Text Length (characters)')
plt.title('Text Length by Label (Box Plot)')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

In [ ]:
# Sample examples from each class
print("\n📝 Sample Examples:\n")
print("="*80)

for label, name in label_map.items():
    print(f"\n🏷️  {name.upper()} (label={label})")
    print("-"*80)
    samples = df[df['label'] == label].sample(min(3, len(df[df['label'] == label])))
    for idx, row in samples.iterrows():
        text = row['text'][:200] + "..." if len(row['text']) > 200 else row['text']
        print(f"\n  • {text}")
    print()

## 5. Data Quality Checks

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Check for duplicates
duplicates = df.duplicated(subset=['text']).sum()
print(f"\nDuplicate text samples: {duplicates}")

# Basic statistics
print("\nText length statistics:")
print(df.groupby('label_name')['text_length'].describe())

## ✅ Phase 1 Complete!

### Summary
- ✅ Downloaded datasets from HuggingFace
- ✅ Labeled data (0=benign, 1=injection, 2=jailbreak)
- ✅ Cleaned text (lowercase, removed URLs/HTML/emojis)
- ✅ Removed duplicates
- ✅ Saved to `data/processed/prompts.csv`

### Next: Phase 2 - Feature Engineering
We'll extract:
- Lexical features (length, keywords, punctuation)
- Structural features (instruction stacking, role switching)
- Semantic features (embeddings, similarity)